In [1]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag


import json
from collections import defaultdict
import operator

import time
import math
import os
import sys
import numpy as np

In [2]:
directory="D:/Hacks/Wehack3/test data/test_nidhish"
namesofdoc=os.listdir(directory)


alpha=0.8
totaldocs=len(namesofdoc)

In [3]:
def read_data(file_name):
    with open(file_name,'r',encoding="utf8") as f:
        word_vocab = set() # not using list to avoid duplicate entry
        word2vector = {}
        for line in f:
            line_ = line.strip() #Remove white space
            words_Vec = line_.split()
            word_vocab.add(words_Vec[0])
            word2vector[words_Vec[0]] = np.array(words_Vec[1:],dtype=float)
    print("Total Words in DataSet:",len(word_vocab))
    return word_vocab,word2vector

In [4]:
totalvocab, w2v = read_data("D:/Deep learning/pretrainedmodels/Glove vectors/glove.6B.50d.txt")

Total Words in DataSet: 400000


In [5]:
json_data=open("D:/Hacks/Wehack3/test data/compresseddict.json").read()

compresseddict = json.loads(json_data)


In [6]:
q="modi India"
q=word_tokenize(q)
q

['modi', 'India']

In [7]:
with open("D:/Hacks/Wehack3/test data/vocab.txt", 'r',encoding='UTF-8') as f:
    vocab = [line.rstrip('\n') for line in f]

In [8]:
for i in range(0,len(q)):
    replace=""
    mindist=1000
    if q[i].lower() not in vocab:
        vec1=w2v[q[i]]
        for j in vocab:
            if j in totalvocab:
                vec2=w2v[j]
                dist = np.linalg.norm(vec1-vec2)
                if dist<mindist:
                    mindist=dist
                    replace=j
        q[i]=replace

In [9]:
x=pos_tag(q)
x

[('modi', 'NNS'), ('India', 'NNP')]

In [10]:
dictofdoc={}

for i in namesofdoc:
    dictofdoc[i]=0

for i in x: 
    info=compresseddict[i[0].lower()]
    #finding number of documents query word is in
    numberofdocs=0
    numofdocsnocontext=0
    docs_counted=[]
    docs_nocontext_counted=[]
    for j in info:     #j has POStag,docn,number
        if j[0]==i[1]: #matches context
            if j[1] not in docs_counted: #to see whether or not the doc is already counted
                docs_counted.append(j[1])
                numberofdocs=numberofdocs+1
        else:
            if j[1] not in docs_counted: #to see whether or not the doc is already counted
                docs_nocontext_counted.append(j[1])
                numofdocsnocontext=numofdocsnocontext+1
            
            
    
    for j in info:     #j has POStag,docn,number
        if j[0]==i[1]: #matches context
            dictofdoc[j[1]]=dictofdoc[j[1]]+(alpha*j[2]*math.log10((totaldocs+1)/numberofdocs))
            
        else: #context not matched
            noncontext_match=j[2]
            dictofdoc[j[1]]=dictofdoc[j[1]]+((1-alpha)*j[2]*math.log10((totaldocs+1)/numofdocsnocontext))   

In [11]:
dictofdoc

{'ainme.txt': 0,
 'choclate.txt': 0,
 'hackathon.txt': 0,
 'Japan.txt': 0,
 'japan_stock.txt': 0,
 'modi-japan-trip.txt': 2.23670401387526,
 'modi.txt': 2.23670401387526,
 'programming.txt': 0,
 'usa.txt': 0}